### Youtube Sentence Pos Tag Analysis

In [5]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [6]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [7]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# data file parameter
file = "200 Classification"  # must be excel file
sheets = "Sheet3"  # "Sheet3"
start_first_column = 0  # start column location number

# classification
threshold_num = 1

In [8]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

#### Part 1

In [9]:
def column_word_list(df, column):
    word_list = []
    for i in df[f"{column}"].dropna():
        words = i.split(",")
        for j in words:
            word = j.strip()
            word_list.append(word)
    return word_list

In [10]:
def column_word(df, column_list):
    '''
    column_word(df_pos_tag_group, df_pos_tag_group.columns) \n
    column_word(df_pos_tag_group, ["col_1","col_2"])
    '''
    word_list_all = []
    for i in column_list:
        word_list = column_word_list(df, f"{i}")
        word_list_all.append(word_list)
    return word_list_all

In [11]:
def classification_condition(list_x, count_condition):
    '''
    from collections import Counter needed
    
    '''
    dict_list_count = Counter(list_x)
    count_list = list(dict_list_count.values())

    if any([True if i>count_condition else False for i in count_list]):
        return "Fail"
    else:
        return "Pass"

    #return any([True if i>count_condition else False for i in count_list])  

In [ ]:
df_pos_tag_group = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/\
Data/{lang_folder.capitalize()}/{file}.xlsx", sheet_name=f"{sheets}")
df_pos_tag_group

In [ ]:
df_youtube_sent_select = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/\
2-Word Group In Youtube Sentence/Turkish English/Turkish_English_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx")
df_youtube_sent_select

In [ ]:
column_word_list_all = column_word(df_pos_tag_group, df_pos_tag_group.columns)

In [ ]:
result_list = []
#for i in df_youtube_sent_select["search_string"]:
for i in range(len(df_youtube_sent_select)):
    #words = word_tokenize(i)
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    words = word_tokenize(search)
    list_var = []
    for j in words:
        a = 0        
        while a < len(column_word_list_all):            
            if j in column_word_list_all[a]:
                list_var.append(a)
            else:
                pass                
            a+=1
    result_list.append([search,start,end,sent,id,id_url,list_var])

In [ ]:
df_youtube_sent_class = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","classification"])
df_youtube_sent_class

In [ ]:
df_youtube_sent_class["condition"] = df_youtube_sent_class["classification"].apply(lambda x: classification_condition(x, threshold_num))
df_youtube_sent_class

In [ ]:
df_youtube_sent_class.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_{threshold_num}_Classification_All.xlsx", index=False)

In [ ]:
df_youtube_sent_class_select = df_youtube_sent_class[df_youtube_sent_class["condition"] == "Pass"]
df_youtube_sent_class_select.reset_index(drop=True, inplace=True)
df_youtube_sent_class_select

In [ ]:
df_youtube_sent_class_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_{threshold_num}_Classification_Select.xlsx", index=False)

#### Part 2

In [12]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [13]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [14]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
195,195,saat,399989
196,196,onunla,399330
197,197,yapıyorsun,398274
198,198,neler,397377


In [15]:
df_youtube_sent_select = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/\
2-Word Group In Youtube Sentence/Turkish English/Turkish_English_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx")
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...
...,...,...,...,...,...,...
120259,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN
120260,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN
120261,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN
120262,o mu o mu,4062,4064,o mu o mu,8cU1k2ylhF4,NaN


In [16]:
df_youtube_sent_select["word_index_list"] = df_youtube_sent_select["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...,"[87, 197, 9, 100, 9, 35, 30, 35, 60, 30, 35, 106]"
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s,"[36, 4, 156, 79, 72, 16, 167, 38, 69, 1, 132, ..."
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...,"[22, 22, 6, 20, 22, 22, 6, 20, 22, 10, 43, 183..."
...,...,...,...,...,...,...,...
120259,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN,"[6, 2, 6, 2]"
120260,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN,"[6, 2, 33, 6]"
120261,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN,"[6, 2, 2, 6]"
120262,o mu o mu,4062,4064,o mu o mu,8cU1k2ylhF4,NaN,"[6, 47, 6, 47]"


In [21]:
#df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List.xlsx", index=False)

In [22]:
df_youtube_sent_select = df_youtube_sent_select[df_youtube_sent_select["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
1,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."
2,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s,"[36, 4, 156, 79, 72, 16, 167, 38, 69, 1, 132, ..."
3,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"
4,şeyler olabilir ne gibi senin gerçekten ben de...,378,383,şeyler olabilir ne gibi senin gerçekten ben de...,YdTPitK_hV0,https://www.youtube.com/watch?v=YdTPitK_hV0&t=...,"[107, 129, 2, 27, 36, 70, 7, 8, 94, 10]"
...,...,...,...,...,...,...,...
36337,mi o da ne,444,445,kabayel mi o da ne,UxIEpobTj6M,NaN,"[5, 6, 15, 2]"
36338,var o da o,416,418,bu yapılarda çok güzel taşları görünüşü bu köy...,mx73zmdFMys,NaN,"[11, 6, 15, 6]"
36339,o da o da,515,517,o da gitti o da o da ikimizde ya işte anlatırk...,bc8xOmoiZm8,NaN,"[6, 15, 6, 15]"
36340,o ne o ne,188,190,o ne o ne,uBDsQXn1DgQ,NaN,"[6, 2, 6, 2]"


In [23]:
df_youtube_sent_select.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36342 entries, 0 to 36341
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   search_string    36342 non-null  object
 1   start_time       36342 non-null  int64 
 2   end_time         36342 non-null  int64 
 3   sentence         36342 non-null  object
 4   video_id         36342 non-null  object
 5   video_url        21216 non-null  object
 6   word_index_list  36342 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.9+ MB


In [27]:
result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]

    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())

    if any([True if i>2 else False for i in count_list]):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])
    

In [28]:
set_index = set(index_list)
set_all_index = set([x for x in range(0,200)])

In [29]:
set_all_index.difference(set_index)

{24,
 37,
 67,
 75,
 78,
 81,
 82,
 92,
 100,
 101,
 102,
 111,
 115,
 118,
 122,
 125,
 127,
 129,
 133,
 136,
 140,
 148,
 149,
 154,
 162,
 163,
 164,
 165,
 168,
 171,
 172,
 185,
 187,
 188,
 196,
 198,
 199}

In [30]:
Counter(index_list)

Counter({36: 2,
         4: 2,
         156: 2,
         79: 2,
         72: 1,
         16: 2,
         167: 2,
         38: 2,
         69: 2,
         1: 2,
         132: 2,
         6: 2,
         9: 2,
         54: 2,
         55: 2,
         70: 2,
         173: 2,
         27: 2,
         12: 2,
         43: 2,
         113: 1,
         2: 2,
         20: 2,
         0: 2,
         83: 2,
         94: 2,
         112: 2,
         73: 2,
         10: 2,
         44: 2,
         30: 2,
         35: 2,
         46: 2,
         14: 2,
         8: 2,
         99: 1,
         7: 2,
         56: 2,
         65: 2,
         3: 2,
         181: 2,
         123: 2,
         174: 2,
         28: 2,
         179: 2,
         25: 2,
         48: 2,
         66: 2,
         85: 1,
         126: 2,
         98: 2,
         86: 2,
         33: 2,
         50: 2,
         52: 2,
         18: 2,
         175: 2,
         74: 1,
         71: 2,
         107: 2,
         49: 1,
         17: 1,
    

In [33]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s,"[36, 4, 156, 79, 72, 16, 167, 38, 69, 1, 132, ..."
1,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"
2,bütün ne kadar bir şey olacak bilmiyorum tam o...,136,139,yani arabayla yapıyoruz bütün ne kadar bir şey...,I5ZZcEVXIME,https://www.youtube.com/watch?v=I5ZZcEVXIME&t=...,"[113, 2, 20, 0, 16, 83, 94, 112, 73, 10, 44]"
3,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
4,bir şeyi ben böyle istiyorum ve bu şekilde ola...,16,21,herhangi bir şeyi ben böyle istiyorum ve bu şe...,CULrx3yerE8,https://www.youtube.com/watch?v=CULrx3yerE8&t=16s,"[0, 99, 7, 56, 65, 3, 1, 181, 83, 123]"
5,evet aslında çok iyi geliyor ama beni şu şekil...,509,512,evet aslında çok iyi geliyor ama beni şu şekil...,Jwei0pKA6NQ,https://www.youtube.com/watch?v=Jwei0pKA6NQ&t=...,"[12, 174, 9, 28, 179, 10, 25, 48, 181, 28]"
6,peki efendim teşekkür ederim ben teşekkür ederim,164,166,peki efendim teşekkür ederim ben teşekkür ederim,HhrBZuxjFxM,https://www.youtube.com/watch?v=HhrBZuxjFxM&t=...,"[66, 85, 126, 98, 7, 126, 98]"
7,o yüzden burada aslında önemli olan şu demek ki,254,258,o yüzden burada aslında önemli olan şu demek k...,_fnQ4MhtREE,https://www.youtube.com/watch?v=_fnQ4MhtREE&t=...,"[6, 132, 38, 174, 156, 79, 48, 86, 33]"
8,hadi biraz daha bakalım belki başka şeyler de,100,104,hadi biraz daha bakalım belki başka şeyler de ...,fvZVXHDNezo,https://www.youtube.com/watch?v=fvZVXHDNezo&t=...,"[50, 52, 18, 175, 74, 71, 107, 8]"
9,misin hayır peki demek ki nerede yapıyorsun,672,675,misin hayır peki demek ki nerede yapıyorsun ol...,narvrRi001U,https://www.youtube.com/watch?v=narvrRi001U&t=...,"[49, 17, 66, 86, 33, 90, 197]"


In [34]:
df_youtube_sent_sample.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx", index=False)

#### Copy Move And Delete

In [35]:
output_file = glob.glob(f"{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_*.xlsx")
output_file

['Turkish_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx']

In [36]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [37]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Custom Pattern

In [ ]:
def classification_order(classification_list, target_pattern_list):
    '''
    classification_order(classification_list, target_pattern_list)
    classification_list and target_pattern_list are list
    '''
    if [x for x in classification_list] == [y for y in target_pattern_list]:
        return "pattern is equal"
    else:
        return "pattern in not equal"

In [ ]:
target_pattern = [7,1,4,1,2,8,7,3,4,1,2,8,7,8,8]

In [ ]:
df_youtube_sent_class["pattern_result"] = df_youtube_sent_class["classification"].apply(lambda x: classification_order(x, target_pattern))
df_youtube_sent_class

In [ ]:
df_youtube_sent_class[df_youtube_sent_class["test"] == "pattern_result"]

In [ ]:
df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Custom_Pattern.xlsx", index=False)

In [ ]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [ ]:
df_youtube_sent_select["word_index_list"] = df_youtube_sent_select["search_string"].apply(lambda x: )

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

In [ ]:
word_index_dict = {key:val for (key, val) in zip(df_word_select["word"],df_word_select["index"])}
#word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word_select["word"])}

In [ ]:
word_index_dict["bu"]

In [ ]:
column_word_list(df_pos_tag_group, "col_1")

In [ ]:
column_word(df_pos_tag_group, df_pos_tag_group.columns)

In [ ]:
df_test = df_youtube_sent_select.head(2)
df_test

In [ ]:
column_word_list_all = column_word(df_pos_tag_group, df_pos_tag_group.columns)

In [ ]:
result_list = []
for i in df_test["search_string"]:
    words = word_tokenize(i)
    list_var = []
    for j in words:
        a = 0        
        while a < len(column_word_list_all):            
            if j in column_word_list_all[a]:
                list_var.append(a)
            else:
                pass                
            a+=1
    result_list.append([i,list_var])

In [ ]:
result_list

In [ ]:
pd.DataFrame(result_list, columns=["sentence","classification"])

In [ ]:
len(column_word_list_all)

In [ ]:
from collections import Counter

In [ ]:
result_list[0][1]

In [ ]:
Counter(result_list[0][1])

In [ ]:
any(Counter(result_list[0][1]).values())

In [ ]:
list_test = list(Counter(result_list[0][1]).values())
list_test

In [ ]:
any([True if i>3 else False for i in list_test])

In [ ]:
def classification_condition(list_x, count_condition):
    '''
    from collections import Counter needed
    
    '''
    dict_list_count = Counter(list_x)
    count_list = list(dict_list_count.values())

    return any([True if i>count_condition else False for i in count_list])   

In [ ]:
list_test2 = result_list[0][1]

In [ ]:
classification_condition(list_test2, 4)